# Projections in Spatial Analysis

Map projections transform the curved surface of the Earth into a flat map, minimizing distortions. 

Without projections, spatial data will be stretched and inaccurate in measurements.

```{figure} ../resources/23.png
:width: 500px
:name: figure-example

Map projection (Source: [randpublishing](https://randpublishing.com/blog/ask-a-cartographer-what-are-some-of-the-coolest-or-most-unusual-map-projections-you-have-come-across/))
````

Coordinate Reference System (CRS) is a system used to define the location of spatial objects on the Earth's surface.

`Geographic Coordinate Systems (GCS)`: Defines locations using latitude and longitude on a spherical model of the Earth (e.g., WGS84).
`Projected Coordinate Systems (PCS)`: Converts the curved surface into a flat map with a Cartesian coordinate system (e.g., UTM).

```{figure} ../resources/24.png
:width: 600px
:name: figure-example

Geographic Coordinate Systems VS Projected Coordinate Systems (Source: [Nature Data Newsletter](https://newsletter.cecil.earth/p/spatial-data-foundations-for-nature))
````

`````{admonition} GCS VS PCS!
:class: note

Geographic Coordinate Systems **(GCS)**: Preserves shape over large areas but can distort distances and areas when flattened. Ideal for **global-scale mapping** where angular relationships matter.

Projected Coordinate Systems **(PCS)**: Reduces distortions for specific regions but may distort shape or scale because flattening a curved surface causes stretching or compression, leading to shape alterations. It is best for **local or regional mapping** that requires accurate measurements.
`````